In [0]:
# Simple CNN model for CIFAR-10
import numpy
#from keras.datasets import cifar10
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras.models import load_model
import keras.callbacks
import tensorboard
K.set_image_dim_ordering('th')


# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# input image dimensions
img_rows, img_cols = 28, 28


# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# normalize inputs from 0-255 to 0.0-1.0
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255.0
x_test = x_test / 255.0
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)


x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)

num_classes = y_test.shape[1]

# Create the model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(1,28,28), activation='relu'))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(Dropout(0.2))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# Fit the model
tbCallBack= keras.callbacks.TensorBoard(log_dir='./Graph', write_images=True)

model.fit(x_train, y_train, validation_data=(x_test, y_test),epochs=2, batch_size=32,  callbacks=[tbCallBack])

model.save('fashionmnist_update.h5')


#loading model
model = load_model('fashionmnist_update.h5')

# Final evaluation of the model
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))



print(x_test.shape)
#Predictions for first four images

for i in range(0,4):
    predicted_value = model.predict(x_test[[i],:])
    predict_classes = model.predict_classes(x_test[[i],:])
    actual_value = y_test[[i],:]
    print("Actual Value for :" + str(i) + 'st Image' + str(actual_value))
    print("Predicted Value for " + str(i) + 'st Image' + str(predict_classes))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 32, 26, 26)        320       
_________________________________________________________________
dropout_19 (Dropout)         (None, 32, 26, 26)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 32, 24, 24)        9248      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 32, 12, 12)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 64, 10, 10)        18496     
_________________________________________________________________
dropout_20 (Dropout)         (None, 64, 10, 10)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 64, 8, 8)          36928     
__________

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-04-19 22:02:37--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.226.180.131, 52.7.169.168, 52.21.103.149, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.226.180.131|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14977695 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  14.28M  59.8MB/s    in 0.2s    

2019-04-19 22:02:37 (59.8 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [14977695/14977695]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
LOG_DIR = './Graph'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://ffef2171.ngrok.io


In [0]:
!pip install keras